In [1]:
from config_004 import *
from subpool_parsing_utils import *
from subpool_plotting_utils import *
from subpool_pdf_report import *

import pandas as pd
import os


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
config_df = pd.read_csv(config_path, sep='\t')
subpools = get_subpools(config_df, plate)

In [ ]:
for subpool in subpools:
    # Load data
    sample_df = parse_sample_df(sample_metadata_path)
    sample_df = sample_df[sample_df['plate'] == plate]

    # Determine color mapping
    color_by = 'Tissue' if sample_df['Tissue'].nunique() > sample_df['Genotype'].nunique() else 'genotype'
    color_dict = get_color_dict(sample_df, color_by)

    # Plot plate map
    plate_map_path = plot_plate_map(sample_df, color_by, color_dict)

    # Alignment stats
    align_df = load_alignment_stats(plate, subpool)

    # QC summary tables
    qc_200umi, qc_500umi = load_qc_stats(plate, subpool, sampletype)

    # Cell counts
    formatted_counts = summarize_cell_counts(qc_200umi, qc_500umi, align_df, sampletype)

    # Generate knee plot
    knee_plot_path = plot_knee_raw_counts(plate, subpool)

    # Load obs table
    adata_obs = pd.read_csv(obs_path)
    adata_obs = adata_obs[adata_obs['subpool'] == subpool]

    # Heatmaps
    hmap_paths = [create_round_heatmap(adata_obs, round_col, kit, sampletype) for round_col in ['bc1_well', 'bc2_well', 'bc3_well']]

    # Cellbender
    cb_results_df = load_cellbender_stats(plate, subpool, sampletype)
    cb_settings_df = load_cellbender_settings(plate, subpool)
    cb_knee_path = plot_knee_cb(adata_obs)

    # Violin plots
    violin1_path, violin2_path = plot_qc_violins(adata_obs)
    adata_obs_filt = filter_obs(adata_obs, min_counts, max_counts, min_genes, pct_counts_mt, doublet_score)
    violin_filt1, violin_filt2 = plot_qc_violins_filtered(adata_obs_filt)

    # Barcode map
    barcode_map_df = create_barcode_sample_map(plate, kit, chemistry)

    # Cell type stacked barplots
    combined_obs = load_combined_obs(sample_df)
    main_tiss = extract_main_tissue(combined_obs, plate)
    multi_tiss = extract_mult_tissue(combined_obs, plate)

    main_celltype_path = plot_stacked_main(combined_obs, subpool, main_tiss, "plots/sample_celltype_proportions_main_subpool.png")
    mult_celltype_path = plot_stacked_mult(combined_obs, subpool, multi_tiss, "plots/sample_celltype_proportions_mult_subpool.png")

    # Create report
    elements = build_pdf_report(
        plate,
        config_df,
        kit,
        chemistry,
        sample_df,
        subpool, # SUBPOOL
        color_by,
        plate_map_path,
        align_df,
        formatted_counts,
        qc_200umi,
        qc_500umi,
        knee_plot_path,
        hmap_paths,
        cb_settings_df,
        cb_results_df,
        cb_knee_path, 
        violin1_path,
        violin2_path,
        violin_filt1,
        violin_filt2,
        main_celltype_path,
        mult_celltype_path,
        barcode_map_df,
        adata_obs,
        adata_obs_filt,
        combined_obs,
        main_tiss,
        multi_tiss,
        sampletype,
        min_counts, 
        min_genes, 
        max_counts, 
        pct_counts_mt, 
        doublet_score
    )

    # Save PDF
    doc = create_pdf_doc(plate, subpool)
    doc.build(elements)
    print(f"Report saved as {doc.filename}")



/tmp/ipykernel_737441/778876920.py:26: DtypeWarning: Columns (24,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(obs_path)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,29,35,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Report saved as igvf_004/igvf_004_Subpool_EX_report.pdf


/tmp/ipykernel_737441/778876920.py:26: DtypeWarning: Columns (24,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(obs_path)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,29,35,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Report saved as igvf_004/igvf_004_Subpool_2_report.pdf


/tmp/ipykernel_737441/778876920.py:26: DtypeWarning: Columns (24,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(obs_path)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,29,35,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


...

/tmp/ipykernel_737441/778876920.py:26: DtypeWarning: Columns (24,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(obs_path)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,29,35,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Report saved as igvf_004/igvf_004_Subpool_14_report.pdf


/tmp/ipykernel_737441/778876920.py:26: DtypeWarning: Columns (24,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(obs_path)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,29,35,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/share/crsp/lab/seyedam/share/8cube_paper/plate_report/subpool_parsing_utils.py:177: DtypeWarning: Columns (24,25,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Report saved as igvf_004/igvf_004_Subpool_15_report.pdf


/tmp/ipykernel_737441/778876920.py:26: DtypeWarning: Columns (24,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(obs_path)
